# **STOKES ELEMENT TESTING**

## **Imports**

In [ ]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

import matplotlib.pyplot as plt
from firedrake.pyplot import tricontourf
import numpy as np

--2025-12-31 17:23:27--  https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4767 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2025-12-31 17:23:28 (45.9 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4767/4767]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ set +x
























#############################################################

# **Code**

In [ ]:
def solve_stokes_eigenproblem(mesh_type="l_shape", refinement_level=4, degree=2, sigma_val=20.0, n_eigenvalues=6, n_eigenfunctions=0):
    """
    Solves the Stokes eigenproblem using a streamfunction formulation.
    
    Parameters:
    - mesh_type: "square" or "l_shape"
    - refinement_level: Number of refinements to apply to the base mesh
    - degree: Polynomial degree for the streamfunction (CG space)
    - n_eigenvalues: Number of eigenvalues to compute
    - sigma_val: Penalty parameter for the biharmonic form
    - plot_eigenfunctions: Whether to display contour plots of the eigenfunctions
    """
    
    # 1. Mesh Definition
    if mesh_type == "square":
        # Base mesh is 1x1
        mesh = UnitSquareMesh(1, 1)
    elif mesh_type == "l_shape":
        coords = np.array([
            [0.0, 0.0],  [1.0, 0.0],  [1.0, 1.0],   [0.0, 1.0],
            [-1.0, 1.0], [-1.0, 0.0], [-1.0, -1.0], [0.0, -1.0]
        ], dtype=float)
        cells = np.array([
            [0, 1, 2], [0, 2, 3],     # top right
            [0, 3, 4], [0, 4, 5],     # top left
            [0, 5, 6], [0, 6, 7],     # bottom left
        ], dtype=np.int32)
        plex = PETSc.DMPlex().createFromCellList(
            2, cells, coords, interpolate=True, comm=PETSc.COMM_WORLD
        )
        mesh = Mesh(plex)
    else:
        raise ValueError(f"Unknown mesh type: {mesh_type}")
    if refinement_level > 0:
        mesh = MeshHierarchy(mesh, refinement_level)[-1]

    print(f"Mesh: {mesh_type}, Refinement Level: {refinement_level}, Cells: {mesh.num_cells()}")

    # 2. Function Spaces
    V = FunctionSpace(mesh, "CG", degree)

    # 3. Trial and Test Functions
    psi = TrialFunction(V)
    phi = TestFunction(V)

    u = curl(psi)
    v = curl(phi)

    # 4. Variational Form
    n = FacetNormal(mesh)
    h = CellDiameter(mesh)
    sigma = Constant(sigma_val)

    def a_biharmonic(u, v, ds_degree=None, penalty_only=False):
        dev = lambda u_ref : sym(grad(u_ref))
        jump_dev = lambda u_ref : 2 * avg(sym(outer(u_ref, n)))

        ds_ = ds if (ds_degree is None) else ds(degree=ds_degree)
        dS_ = dS if (ds_degree is None) else dS(degree=ds_degree)

        if penalty_only:
            term = 2 * inner(jump_dev(u), jump_dev(v)) * dS_
            term += 2 * inner(sym(outer(u, n)), sym(outer(v, n))) * ds_
        else:
            # Volume term: 2 * dev(u) : dev(v)
            term = 2 * inner(dev(u), dev(v)) * dx

            # Interior Facets
            term -= 2 * inner(avg(dev(u)), jump_dev(v)) * dS_
            term -= 2 * inner(avg(dev(v)), jump_dev(u)) * dS_
            term += 2 * (sigma / avg(h)) * inner(jump_dev(u), jump_dev(v)) * dS_

            # Boundary Facets (Weak imposition of no-slip BC u=0)
            term -= 2 * inner(1/2 * dev(u), sym(outer(v, n))) * ds_
            term -= 2 * inner(1/2 * dev(v), sym(outer(u, n))) * ds_
            term += 2 * (sigma / h) * inner(sym(outer(u, n)), sym(outer(v, n))) * ds_

        return term

    A = a_biharmonic(u, v)
    M = inner(u, v) * dx

    # 5. Solver
    # Boundary conditions: psi = 0 on boundary (u.n = 0)
    bcs = [DirichletBC(V, 0, "on_boundary")]

    eigenproblem = LinearEigenproblem(A, M, bcs=bcs)

    solver = LinearEigensolver(eigenproblem, n_eigenvalues, solver_parameters={
        "eps_type": "krylovschur",
        "eps_target": 0.0,
        "eps_which": "TARGET_MAGNITUDE",
        "st_type": "sinvert",
        "st_pc_type": "lu",
        "st_pc_factor_mat_solver_type": "mumps"
    })

    n_found = solver.solve()
    print(f"Found {n_found} eigenvalues.")

    # 6. Output & Plotting
    for i in range(n_found):
        lam = solver.eigenvalue(i)
        print(f"Eigenvalue {i}: {lam}")

    n_eigenfunctions = min(n_eigenfunctions, n_found)
    if n_eigenfunctions > 0:
        # Determine grid size for subplots
        cols = 3
        rows = (n_eigenfunctions + cols - 1) // cols
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
        if n_eigenfunctions == 1:
            axes = [axes]
        else:
            axes = axes.flatten()

        for i in range(n_eigenfunctions):
            psi_mode_real, _ = solver.eigenfunction(i)
            
            # Plot streamfunction
            plt.sca(axes[i])
            tricontourf(psi_mode_real, axes=axes[i])
            axes[i].set_title(f"Mode {i+1}: $\lambda$={solver.eigenvalue(i):.4f}")
            axes[i].set_aspect('equal')
            
        # Hide unused subplots
        for i in range(n_found, len(axes)):
            axes[i].axis('off')
            
        plt.tight_layout()
        plt.show()

In [ ]:
# Run the solver with default parameters (L-shape)
solve_stokes_eigenproblem(mesh_type="l_shape", refinement_level=4, degree=2, sigma_val=20.0, n_eigenvalues=6, n_eigenfunctions=0)